In [3]:
import tensorflow as tf

In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [100]:
m = 10
n_z = 2
n_epoch = 10

In [101]:
# Encoder Part
inputs = tf.keras.layers.Input(shape=(784, ))
h_q = tf.keras.layers.Dense(512, activation="relu")(inputs)
mu = tf.keras.layers.Dense(n_z, activation="linear")(h_q)
log_sigma = tf.keras.layers.Dense(n_z, activation="linear")(h_q)

In [102]:
def sample_z(args):
    mu, log_sigma = args
    eps = tf.keras.backend.random_normal(shape=(m, n_z), mean=0., stddev=1.)
    return mu + tf.keras.backend.exp(log_sigma / 2) * eps

In [103]:
# Sampling
z = tf.keras.layers.Lambda(sample_z)([mu, log_sigma])

In [104]:
# Decoder Part
decoder_hidden = tf.keras.layers.Dense(512, activation="relu")
decoder_out = tf.keras.layers.Dense(784, activation="sigmoid")

In [105]:
h_p = decoder_hidden(z)
outputs = decoder_out(h_p)

In [106]:
vae = tf.keras.models.Model(inputs, outputs)

In [107]:
encoder = tf.keras.models.Model(inputs, mu)

In [108]:
d_in = tf.keras.layers.Input(shape=(n_z,))
d_h = decoder_hidden(d_in)
d_out = decoder_out(d_h)
decoder = tf.keras.models.Model(d_in, d_out)

In [109]:
def vae_loss(y_true, y_pred):
    recon = tf.keras.backend.sum(tf.keras.backend.binary_crossentropy(y_pred, y_true), axis=1)
    kl = 0.5 * tf.keras.backend.sum(tf.keras.backend.exp(log_sigma) + tf.keras.backend.square(mu) - 1 - log_sigma, axis=1)
    
    return recon + kl

In [110]:
vae.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.01), loss=vae_loss)

In [71]:
from tensorflow.examples.tutorials.mnist import input_data

In [22]:
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

In [34]:
mnist_images.shape

(60000, 28, 28)

In [58]:
dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(mnist_images.reshape(mnist_images.shape[0], -1)[:100, :] / 255, tf.float32),
     tf.cast(mnist_images.reshape(mnist_images.shape[0], -1)[:100, :] / 255, tf.float32))
)

In [59]:
dataset

<TensorSliceDataset shapes: ((784,), (784,)), types: (tf.float32, tf.float32)>

In [60]:
dataset = dataset.shuffle(1000).batch(32)

In [61]:
dataset

<BatchDataset shapes: ((?, 784), (?, 784)), types: (tf.float32, tf.float32)>

In [111]:
X_train = np.random.rand(10, 784)

In [112]:
X_train.shape

(10, 784)

In [114]:
#tf.initialize_all_variables()
vae.fit(X_train, X_train, epochs=20)

Epoch 1/20
10/10 [==============================] - 0s 554us/step - loss: 736.2656
Epoch 2/20
10/10 [==============================] - 0s 688us/step - loss: 724.8429
Epoch 3/20
10/10 [==============================] - 0s 656us/step - loss: 710.5980
Epoch 4/20
10/10 [==============================] - 0s 592us/step - loss: 724.1906
Epoch 5/20
10/10 [==============================] - 0s 729us/step - loss: 730.4393
Epoch 6/20
10/10 [==============================] - 0s 1ms/step - loss: 795.1268
Epoch 7/20
10/10 [==============================] - 0s 633us/step - loss: 834.5702
Epoch 8/20
10/10 [==============================] - 0s 652us/step - loss: 822.4849
Epoch 9/20
10/10 [==============================] - 0s 570us/step - loss: 775.6234
Epoch 10/20
10/10 [==============================] - 0s 588us/step - loss: 729.5927
Epoch 11/20
10/10 [==============================] - 0s 605us/step - loss: 708.4590
Epoch 12/20
10/10 [==============================] - 0s 791us/step - loss: 737.4695
Epo

In [ ]:
b_model = tf.keras.models.Sequential(
    tf.keras.layers.Dense(16, input_shape=(50,), activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
)

b_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    loss="mse",
    #metrics=[""]
)

